In [1]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
import tempfile

In [2]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [3]:
tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [4]:
application_description = """
generate a bot that greets me
""".strip()

In [5]:
my_bot = application.create_bot(application_description)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Compiling TypeSpec...
Compiling Typescript Schema Definitions...
Compiling Drizzle...
Generating Router...
Compiling Handlers...
Generating Application...


In [6]:
my_bot

ApplicationOut(typespec=TypespecOut(reasoning='For a greeting bot, we need to handle basic greeting functionality. The main function would be to generate a greeting based on:\n1. User\'s name (if provided)\n2. Time of day (morning/afternoon/evening)\n3. Optional custom message\n\nThe LLM can extract these from natural language messages like:\n"Hi, I\'m John" -> greet({name: "John"})\n"Good morning" -> greet({timeOfDay: "morning"})\n"Say hi to Mary with a special message" -> greet({name: "Mary", message: "special message"})\n\nThe greeting function should take into account 1 previous message for context.', typespec_definitions='model GreetingRequest {\n    name?: string;\n    timeOfDay?: string;\n    message?: string;\n}\n\ninterface GreetingBot {\n    @llm_func(1)\n    greet(request: GreetingRequest): string;\n}', llm_functions=['greet'], error_output=None), drizzle=DrizzleOut(reasoning="The application has a greeting system with:\n1. A GreetingRequest model that contains optional fiel

In [7]:
my_bot.gherkin

GherkinOut(reasoning=None, gherkin=None, error_output=None)

In [8]:
application.generation_dir

'/tmp/tmp7sme5xwd/generated/generation-1'

In [9]:
!pushd {application.generation_dir}/app_schema
!npm install
!npx tsc src/main.ts
!popd


/tmp/tmp7sme5xwd/generated/generation-1/app_schema ~/neon/bot-new-1/agent


npm ERR! code ENOENT
npm ERR! syscall open
npm ERR! path /home/eugenek/neon/bot-new-1/agent/package.json
npm ERR! errno -2
npm ERR! enoent ENOENT: no such file or directory, open '/home/eugenek/neon/bot-new-1/agent/package.json'
npm ERR! enoent This is related to npm not being able to find a file.
npm ERR! enoent 

npm ERR! A complete log of this run can be found in:
npm ERR!     /home/eugenek/.npm/_logs/2025-02-17T12_21_03_124Z-debug-0.log

                                                                               
                This is not the tsc command you are looking for                
                                                                               

To get access to the TypeScript compiler, tsc, from the command line either:

- Use npm install typescript to first add TypeScript to your project before using npx
- Use yarn to avoid accidentally running code from un-installed packages
/bin/bash: line 1: popd: directory stack empty


In [10]:
typespec = application._make_typespec(application_description)
typespec

TypespecOut(reasoning='For a greeting bot, I expect:\n1. The bot should be able to greet users with personalized messages\n2. The greeting should be contextual based on time of day (morning/afternoon/evening)\n3. LLM can extract name and potentially preferred greeting style from messages like:\n   "Hi, I\'m John" -> greet({name: "John"})\n   "Hello, call me Sarah" -> greet({name: "Sarah"})\nThe interface should be simple as the main functionality is just greeting.\nWe\'ll need:\n- greet function that takes name and returns appropriate greeting\n- The time of day can be inferred by the system, so no need for explicit time parameter', typespec_definitions='model GreetingRequest {\n    name: string;\n    style?: string;  // formal/casual/friendly etc.\n}\n\ninterface GreetingBot {\n    @llm_func(1)\n    greet(request: GreetingRequest): string;\n}', llm_functions=['greet'], error_output=None)

In [11]:
typescript_schema = application._make_typescript_schema(typespec.typespec_definitions)
typescript_schema

TypescriptOut(reasoning="The application consists of:\n1. A request model for greetings that includes a required name and an optional style\n2. An interface for a greeting bot that processes these requests\nThe optional parameter in TypeSpec is denoted with '?', which translates directly to TypeScript syntax", typescript_schema="export interface GreetingRequest {\n    name: string;\n    style?: string;\n}\n\n// Note: The @llm_func decorator from TypeSpec is typically not needed in the TypeScript\n// implementation, as it's more of a runtime/implementation detail\nexport interface GreetingBot {\n    greet(request: GreetingRequest): Promise<string>;\n}", type_names=['GreetingRequest', 'GreetingBot'], error_output=None)

In [12]:
drizzle = application._make_drizzle(typespec.typescript_schema)
drizzle

AttributeError: 'TypespecOut' object has no attribute 'typescript_schema'

In [ ]:
drizzle.error_output, drizzle.reasoning, drizzle.drizzle_schema

In [ ]:
router = application._make_router(typespec.data.output.typespec_definitions)
router.score, router.data.output

In [ ]:
handlers = application._make_handlers(
    typespec.data.output.llm_functions,
    typespec.data.output.typespec_definitions,
    typescript_schema.data.output.typescript_schema,
    drizzle.data.output.drizzle_schema,
)

In [ ]:
for name, handler in handlers.items():
    print(name, handler.score, handler.depth, handler.data.output)
    if handler.score != 1:
        print(handler.data.output.feedback["stdout"])